In [1]:
### import required libraries
import logging
logging.basicConfig(level=logging.DEBUG)
import json
import requests

In [2]:
res_list = [{'app_success': "we couldn't find any revelant clients from the nyt news."},
            {'app_success': "we couldn't find any revelant clients from the wsj news."},
            {'app_success': "we couldn't find any revelant clients from the reuters news."},
            {'app_success': "we couldn't find any revelant clients from the middlemkt news."}]

In [3]:
res_list = [{'app_success': "we couldn't find any revelant clients from the nyt news."},
            {'app_success': "we couldn't find any revelant clients from the wsj news."},
            {'app_success': "we couldn't find any revelant clients from the reuters news."},
            {'app_success': "success", 
            'records': [{'company':'jepg', 'broker': "xxx", "broker_email": "xxx@cbre.com", "news": [["jepg acquired xxx",'www.link.com'], ["jepg acquired xxx2",'www.link.com']]}]}]

In [4]:
res_list = [{'app_success': "we couldn't find any revelant clients from the nyt news."},
            {'app_success': "success", 'records': [{'company':'jepg', 'broker': "xxx", "broker_email": "xxx@cbre.com", "news": [["jepg acquired xxx3",'www.link.com']]}, 
                                                   {'company':'apple', 'broker': "xxx", "broker_email": "xxx@cbre.com", "news": [["apple acquired xxx3", 'www.link.com']]}]},
            {'app_success': "we couldn't find any revelant clients from the reuters news."},
            {'app_success': "success", 
            'records': [{'company':'jepg', 'broker': "xxx", "broker_email": "xxx@cbre.com", "news": [["jepg acquired xxx", 'www.link.com'], ["jepg acquired xxx2", 'www.link.com']]}]}]

In [5]:
def slack_json_builder(slack_dict):

    message = {
        "blocks": [
                    {
                        "type": "section",
                        "text": {
                            "type": "plain_text",
                            "text": slack_dict['message']
                        }
                    }
                ]
            }

    if 'content' in slack_dict.keys():

        content_list = slack_dict['content']
        
        for each_content in content_list:

            message['blocks'].append({'type':'divider'})
            
            one_block = list(each_content.values())

            company_info = one_block[:3]
            company_info[0] = "*" + company_info[0] + "*"
            news = one_block[3]

            num_space = (len(company_info) - len(news))
            num_space_list = [' '] * num_space

            if num_space>0:
                news.extend(num_space_list)
            elif num_space<0:
                company_info.extend(num_space_list)
            
            fields = []
            
            for i in range(len(company_info)):
                fields.append({"type":"mrkdwn", "text": company_info[i]})

                if news[i] != ' ':
                    fields.append({"type":"mrkdwn", "text": "<"+ news[i][1] + "|" + news[i][0] + ">"})
                else:
                    fields.append({"type":"mrkdwn", "text": news[i]})
                    
            message['blocks'].append({"type": "section", "fields":fields})

    return message

In [6]:
def talk_to_slack(slack_json, link):
    
    r = requests.post(url = link, 
                      json = slack_json,
                      headers={'Content-type':'application/json'})
                      
    return 'sent message to slack'

In [7]:
def slack_error(message):
    slack_dict = {}
    slack_dict['message'] = message
    slack_json = slack_json_builder(slack_dict)

    with open("../news_scrapper/combinor/slackchannel.json") as f:  
        links = json.load(f)
        
    link = links[0]['error_report']

    talk_to_slack(slack_json, link)

    return 'report error success'


In [19]:
slack_dict = {'company':'abc', 'news':['aaa','bbb'], 'link': ['aaa.com', 'bbb.com']}
slack_dict['combo'] = list(zip(slack_dict['news'], slack_dict['link']))

del slack_dict['news']
del slack_dict['link']

In [8]:
def slack_success(slack_dict):

    slack_json = slack_json_builder(slack_dict)

    with open("../news_scrapper/combinor/slackchannel.json") as f:  
        links = json.load(f)

    link = links[0]['news_report']

    talk_to_slack(slack_json, link)
    
    return 'report news success'

In [11]:
# for result in res_list:
#     if result['app_success']!='success':
        # slack_error(result['app_success'])

have_results  = [result for result in res_list if result['app_success']== 'success']

if have_results !=[]:
    slack_dict = {'content':[]}

    for each_web_result in have_results:
        for each_record in each_web_result['records']:
            if slack_dict['content'] !=[]:
                if each_record['company'] in [i['company'] for i in slack_dict['content']]:
                    for i in slack_dict['content']:
                        if each_record['company'] == i['company'] and each_record['news'] not in i['news']:
                            i['news'].extend(each_record['news'])
                else:
                    slack_dict['content'].append(each_record)
            else:
                slack_dict['content'].append(each_record)

    slack_dict['message'] = 'Here are the important m&a news for the past 1 week.'
    # slack_success(slack_dict)


In [13]:
with open("../news_scrapper/combinor/slackchannel.json") as f:  
    links = json.load(f)

link = links[0]['news_report']

talk_to_slack(slack_json_builder(slack_dict), link)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): hooks.slack.com:443
DEBUG:urllib3.connectionpool:https://hooks.slack.com:443 "POST /services/T8T67PZAQ/B034Q9R6THD/nSeUQTDWQEeAcyYaneKHrU75 HTTP/1.1" 200 22


'sent message to slack'